<img style="float: right;" src="misc/logo.png" width= "200" height = "200">

#  Data integration with Python 🐍
_____________

This exercise aims to introduce loading, plotting and integrating core data with Python. We will load XRF data and an image with Python, and then plot them together. These are common data types collected at BOSCORF.

____________

**TIPS**

* Press 'shift + enter' or 'shift + command' to run a cell, or press 'Cell' menu on the top bar
* Click 'b' to make a new cell, or press 'Insert' menu on the top bar 
* To clear the whole notebook and its outputs press 'Kernel' then 'Restart and Clear Output'

____________

### 1. Import functions 📚

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from boscode import *

%config InlineBackend.figure_format='retina'
plt.rcParams.update({'font.size': 12})
pd.set_option('display.max_columns', None)

### 1. Load and plot XRF ⚡⏳

First we will load the XRF data, using an example from the Whittard Canyon. We will load the data from Excel to a Pandas Dataframe, which is like Excel but for Python.

Run the following cell:

In [ ]:
## Run this cell to load the data
xrf = load_xrf(path = 'data/itrax/JC36/JC36_74_4/JC36_74_4/Results.txt')

In [ ]:
# creates a new columns with position in metres
xrf['position (m)'] = xrf['position (mm)']/1000

**We can then plot the data using Matplotlib, a library used for data visualisation in Python**

* to select different elements ('data') run in a cell:

    **xrf.columns**
    
* to select different colours follow this link:

    https://matplotlib.org/stable/gallery/color/named_colors.html

* to select different markers follow this link:

    https://matplotlib.org/stable/api/markers_api.html
    
    
 * to zoom in on a section change 'ylim'

In [ ]:
xrf.columns

In [ ]:
data = ['Si', 'Ca']

colors = ['red', 'green']

linewidth = 1

marker = None

markersize = 0

ylim = (1.2, 0)

##################

fig, ax = plt.subplots(ncols = len(data), figsize = (len(data) * 2, 8))

for ax_no, i in enumerate(zip(data, colors)):

    ax[ax_no].plot(xrf[i[0]], xrf['position (mm)']/1000, color = i[1], 
                   marker = marker, markersize = markersize, linewidth = linewidth)
    ax[ax_no].set(ylim = ylim, xlabel = 'counts', title = i[0])
    ax[ax_no].ticklabel_format(axis='x', style='sci', scilimits=(0,0))
    ax[ax_no].xaxis.labelpad = 20 
    
ax[0].set(ylim = ylim, xlabel = 'counts', ylabel = 'depth of section [m]')

plt.tight_layout(w_pad = 2)

**We can also use cross plots (scatter plots) to see relationships between data**

* to select different colour maps follow this link:

    https://matplotlib.org/stable/tutorials/colors/colormaps.html


In [ ]:
data1 = ['Si', 'Si']
data2 = ['Ca', 'Fe']

colour_map = ['magma', 'viridis']

marker = 'o'

markersize = 10

figsize = (7, 4)

##################

fig, ax = plt.subplots(ncols = len(data1), figsize = figsize)

for ax_no, i in enumerate(zip(data1, data2, colour_map)):

    ax[ax_no].scatter(xrf[i[0]], xrf[i[1]], c = xrf[i[1]], edgecolor = 'k',
                   lw = 0.2, marker = marker, s = markersize, cmap = i[2])
    ax[ax_no].set(xlabel = f'{i[0]} [counts]', ylabel = f'{i[1]} [counts]')
    ax[ax_no].ticklabel_format(axis='x', style='sci', scilimits=(0,0))
    ax[ax_no].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax[ax_no].xaxis.labelpad = 20 
    ax[ax_no].set_title(label = f'{i[0]} v {i[1]}', pad = 20)

plt.tight_layout(w_pad = 1)

### 2. Load and plot image and xray 📷 ⏳

You can also load images directly

In [ ]:
image_path = f'data/images/JC036_74_4/IM001_01.tif'

core_image = plt.imread(image_path)

plt.figure(figsize = (2, 7))
plt.imshow(core_image, aspect = 'auto')
# plt.imshow(core_image[:-325], aspect = 'auto')

The functions below do some work in the background to help out

* Use 'ylim_upper' and 'ylim_lower' to zoom in the core

* Use 'xray_min' and 'xray_max' to change the xray contrast

In [ ]:
core_df, core, images_shp = load_image(74, 4)
xray, lam = load_xray(74, 4)

In [ ]:
ylim_upper = 0.8
ylim_lower = 1.4

xray_min = 0.58
xray_max = 0.65

###############

fig, ax = plt.subplots(figsize = (6, 8), ncols = 2)

ax[0].imshow(core, aspect = 'auto')
ax[1].imshow(lam, aspect = 'auto', cmap = 'gray', vmin = xray_min, vmax = xray_max)

for ax in ax:

    ax.set(yticks = np.arange(0, len(core), 1000), 
          yticklabels = np.arange(0, len(core), 1000)/(5000), 
           ylabel = '[m]', ylim = (ylim_lower * 5000, ylim_upper * 5000))

plt.tight_layout()

### 3. Integrate image and x-ray and XRF 📷 ⏳

Now we can integrate our image and x-ray data to start making interpretations 

* Change the element in 'data' to check how different elements relate to the stratigraphy 

In [ ]:
data = ['Si']

ylim_upper = 0.
ylim_lower = 1.1

#######################

fig, ax = plt.subplots(figsize = (6, 8), ncols = 3)

ax[0].imshow(core, aspect = 'auto')
ax[0].set(title = 'image')
ax[0].set(yticks = np.arange(0, len(core), 1000), 
          yticklabels = np.arange(0, len(core), 1000)/5000,
         ylabel = 'depth of section [m]') 
ax[0].set(yticks = np.arange(0, len(core), 1000), 
          yticklabels = np.arange(0, len(core), 1000)/(5000), 
           ylabel = '[m]', ylim = (ylim_lower * 5000, ylim_upper * 5000))

ax[1].imshow(lam, aspect = 'auto', cmap = 'gray', vmin = 0.58, vmax = 0.65)
ax[1].set(title = 'xray')
ax[1].set(yticks = np.arange(0, len(core), 1000), 
          yticklabels = np.arange(0, len(core), 1000)/(5000), 
           ylabel = '[m]', ylim = (ylim_lower * 5000, ylim_upper * 5000))
ax[1].axes.get_xaxis().set_visible(False)
ax[1].axes.get_yaxis().set_visible(False)

ax[2].plot(xrf[data], (xrf['position_corr']/1000) - 0.02, color = 'k', lw = 1)
ax[2].set(ylim = (ylim_lower, ylim_upper), xlim = (0), title = data[0])
ax[2].axes.get_yaxis().set_visible(False)

plt.tight_layout()